**MUSIC RECOMMENDATION SYSTEM (MACHINE LEARNING)**

# Importing Libraries

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Datasets

In [37]:
vet_df = pd.read_csv('/content/drive/MyDrive/Machine Learning Projects/Music Recommendation System/Million Song Dataset/kaggle_visible_evaluation_triplets.txt',sep='\t',names=['user_id','Songs ID','Frequency'])

unique_df = pd.read_csv('/content/drive/MyDrive/Machine Learning Projects/Music Recommendation System/Million Song Dataset/unique_tracks.txt',sep='<SEP>',names=['Track ID','Songs ID','Composer','Song Name'])

# Exploratory Data Analysis


### Visible Evaluation Triplets Dataset 

It contains User ID, Songs ID, Frequency

In [38]:
vet_df.head()

,user_id,Songs ID,Frequency
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1


In [39]:
vet_df.describe(include= 'all')

,user_id,Songs ID,Frequency
count,1450933,1450933,1.450933e+06
unique,110000,163206,NaN
top,7d90be8dfdbde170f036ce8a4b915440137cb11c,SOFRQTD12A81C233C0,NaN
freq,53,5043,NaN
mean,NaN,NaN,3.187149e+00
std,NaN,NaN,7.051664e+00
min,NaN,NaN,1.000000e+00
25%,NaN,NaN,1.000000e+00
50%,NaN,NaN,1.000000e+00
75%,NaN,NaN,3.000000e+00


In [40]:
vet_df.isnull().sum()

user_id      0
Songs ID     0
Frequency    0
dtype: int64

In [41]:
print("Shape of Visible evaluation triplets is {}".format(vet_df.shape))

Shape of Visible evaluation triplets is (1450933, 3)


### Unique Tracks Dataset

It contains Track ID, Songs ID, Composer, Song Name

In [42]:
unique_df.head()

,Track ID,Songs ID,Composer,Song Name
0,TRMMMYQ128F932D901,SOQMMHC12AB0180CB8,Faster Pussy cat,Silent Night
1,TRMMMKD128F425225D,SOVFVAK12A8C1350D9,Karkkiautomaatti,Tanssi vaan
2,TRMMMRX128F93187D9,SOGTUKN12AB017F4F1,Hudson Mohawke,No One Could Ever
3,TRMMMCH128F425532C,SOBNYVR12A8C13558C,Yerba Brava,Si Vos Querés
4,TRMMMWA128F426B589,SOHSBXH12A8C13B0DF,Der Mystic,Tangle Of Aspens


In [43]:
unique_df.drop('Track ID',axis=1,inplace=True)

In [44]:
unique_df.describe(include='all')

,Songs ID,Composer,Song Name
count,1000000,1000000,999985
unique,999056,72665,702000
top,SOIZVRY12A8151CDDE,Michael Jackson,Intro
freq,3,194,1511


In [45]:
print("Shape of Visible evaluation triplets is {}".format(vet_df.shape))

Shape of Visible evaluation triplets is (1450933, 3)


In [46]:
unique_df['Songs ID'].duplicated().sum()

944

In [47]:
unique_df.isnull().sum()

Songs ID      0
Composer      0
Song Name    15
dtype: int64

In [48]:
unique_df['Song Name'] = unique_df['Song Name'].fillna(unique_df['Song Name'].mode()[0])

In [49]:
unique_df.isnull().sum()

Songs ID     0
Composer     0
Song Name    0
dtype: int64

## Merging the Datasets

In [50]:
combined = pd.merge(vet_df,unique_df,on='Songs ID')

In [51]:
len(combined)

1491952

In [52]:
combined.head(5)

,user_id,Songs ID,Frequency,Composer,Song Name
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,Dwight Yoakam,You're The One
1,c34670d9c1718361feb93068a853cead3c95b76a,SOBONKR12A58A7A7E0,1,Dwight Yoakam,You're The One
2,c5006d9f41f68ccccbf5ee29212b6af494110c5e,SOBONKR12A58A7A7E0,1,Dwight Yoakam,You're The One
3,e4332e11f4df6dd26673bb6b085e9a2bbdc9b8a5,SOBONKR12A58A7A7E0,2,Dwight Yoakam,You're The One
4,baf2fe5885ab93fbbdb7fecc6691788e70afb6c8,SOBONKR12A58A7A7E0,4,Dwight Yoakam,You're The One


In [53]:
combined.tail()

,user_id,Songs ID,Frequency,Composer,Song Name
1491947,9d2f78e2bda5a004879d3ee53ee7de5cdf99c730,SOIDWNE12A8C13E669,1,Our Broken Garden,The Blinding
1491948,987b654b09b239f7c47751e0cfaa2990834cbb55,SOATNWN12A6D4F826F,1,Emilio,Como A Nadie
1491949,987b654b09b239f7c47751e0cfaa2990834cbb55,SOGMCFE12AB018A3A1,3,Morbid Angel,Victorious March Of Reign The Conqueror
1491950,fc072bde2043756ea30ff07e4a5311e34825b4df,SOBAOTU12A6D4F7A68,1,Gus Gus,Call of the Wild
1491951,fc072bde2043756ea30ff07e4a5311e34825b4df,SOGWSIO12A8C133A5B,1,Gus Gus,Snoozer


## Total no of Users, Songs and Composers


In [54]:
no_of_users = len(pd.unique(combined['user_id']))
print('Total no of Unique Users:',no_of_users)

Total no of Unique Users: 110000


In [55]:
no_of_songs = len(pd.unique(combined['Songs ID']))
print('Total no of Unique Songs:',no_of_songs)

Total no of Unique Songs: 163206


In [56]:
unique_artists = len(pd.unique(combined['Composer']))
print('Total no of Unique Artists:',unique_artists)

Total no of Unique Artists: 28367


In [73]:
users = combined['user_id'].unique()
len(users)

110000

## Total Unique no of Songs Listened by per User

In [57]:
  Songs_per_user = combined.groupby('user_id')['Songs ID'].count()
  Songs_per_user

user_id
00007a02388c208ea7176479f6ae06f8224355b3     9
00014a76ed063e1a749171a253bca9d9a0ff1782    11
00015189668691680bb1a2e58afde1541ec92ced    17
0001ff7aa2667c8d8b945317b88adaed1c0b9dc2     9
00020fcd8b01986a6a85b896ccde6c49f35142ad    32
                                            ..
fffdef71f13352e9cff769f1d96f5ccf90f8955e     7
fffe29116f96c97b47a5dabdd406784ad0ba6f30     6
fffe5b73c50c72ca9c54947efac1fcacf59c4a17    23
fffed0bee753cd78494011b8b55dafd4f5c7e5ff    13
ffff07d7d9bb187aa58c7b81b3d3f35e7cf7c0ee    13
Name: Songs ID, Length: 110000, dtype: int64

In [58]:
Songs_per_user.describe()

count    110000.000000
mean         13.563200
std           8.321648
min           5.000000
25%           7.000000
50%          11.000000
75%          18.000000
max          57.000000
Name: Songs ID, dtype: float64

In [59]:
Songs_per_user.sort_values(ascending=False).head()

user_id
316110734d8da7478cc33237458814f770a9eb7a    57
ad4b2717e89766b66b96fe52a38736116e315874    57
d30e18323f15426c3cdc8585252ed34459916f51    55
3233c598c50f0ddbd351504e773cd51de79691db    55
016a24e91a72c159a5048ab1b9b2ba5ce761b526    54
Name: Songs ID, dtype: int64

In [60]:
Songs_per_user.sort_values(ascending=True).head()

user_id
b4f681dac463c297568bffe7f4b646b74a5e6d0d    5
1691b9058739f0b88ac2777e5bbffe0e1d400f2d    5
d1764e1f4626164793d400b95efb4a5f80a06cf9    5
d1776bb80cf8298288974c52f903cc3b9601761a    5
98d00906e235bede8d2b14b9cf5b802ee2438231    5
Name: Songs ID, dtype: int64

# Recommendation Engine

## Popularity based Recommendation Engine

In [74]:
combined['Artist Name - Song Name'] = combined["Composer"] + ' - ' + combined["Song Name"]
song_grouped = combined.groupby(['Artist Name - Song Name']).agg({'Frequency': 'count'}).reset_index()
grouped_sum = song_grouped['Frequency'].sum()
song_grouped['percentage']  = song_grouped['Frequency'].div(grouped_sum)*100
song_grouped.sort_values(['Frequency', 'Artist Name - Song Name'], ascending = [0,1])

,Artist Name - Song Name,Frequency,percentage
59898,Harmonia - Sehr kosmisch,5043,0.338014
17182,Björk - Undo,4483,0.300479
42724,Dwight Yoakam - You're The One,4136,0.277221
77698,Kings Of Leon - Use Somebody,3822,0.256174
51343,Florence + The Machine - Dog Days Are Over (Ra...,3780,0.253359
...,...,...,...
162095,Ólafur Arnalds - Himininn er að hrynja_ en stj...,1,0.000067
162097,Ólafur Arnalds - Kjurrt,1,0.000067
162104,Ólafur Arnalds - Við vorum smá...,1,0.000067
162108,Órla Fallon - The Water Is Wide,1,0.000067


In [75]:
import sklearn
from sklearn.model_selection import train_test_split 
train_data, test_data = train_test_split(combined, test_size = 0.20, random_state=0)

In [76]:
#Class for Popularity based Recommender System model
class Popularity_Recommendation_engine():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None
        
    #Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

        #Get a count of user_ids for each unique song as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped.rename(columns = {'user_id': 'score'},inplace=True)
    
        #Sort the songs based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])
    
        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)

    #Use the popularity based recommender system model to make recommendations
    def recommend(self, user_id):    
        user_recommendations = self.popularity_recommendations
        
        #Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id
    
        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        
        return user_recommendations

In [77]:
pm = Popularity_Recommendation_engine()
pm.create(train_data, 'user_id', 'Artist Name - Song Name')
user_id = users[8]
pm.recommend(user_id)

,user_id,Artist Name - Song Name,score,Rank
54790,daa9e7e53ae787ab4f1b5518b695198947d821a2,Harmonia - Sehr kosmisch,4072,1.0
15799,daa9e7e53ae787ab4f1b5518b695198947d821a2,Björk - Undo,3546,2.0
39135,daa9e7e53ae787ab4f1b5518b695198947d821a2,Dwight Yoakam - You're The One,3346,3.0
47004,daa9e7e53ae787ab4f1b5518b695198947d821a2,Florence + The Machine - Dog Days Are Over (Ra...,3057,4.0
71079,daa9e7e53ae787ab4f1b5518b695198947d821a2,Kings Of Leon - Use Somebody,3041,5.0
71072,daa9e7e53ae787ab4f1b5518b695198947d821a2,Kings Of Leon - Revelry,2986,6.0
95430,daa9e7e53ae787ab4f1b5518b695198947d821a2,OneRepublic - Secrets,2742,7.0
12282,daa9e7e53ae787ab4f1b5518b695198947d821a2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,2615,8.0
68074,daa9e7e53ae787ab4f1b5518b695198947d821a2,Justin Bieber - Somebody To Love,2596,9.0
138870,daa9e7e53ae787ab4f1b5518b695198947d821a2,Train - Hey_ Soul Sister,2233,10.0
